### Baseline experiment

Experiment to compare the 4 Optimization algorithms before trying to improve sparse search

In [70]:
import sys  
sys.path.insert(0, '../')

import HPO
import pysgpp
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import KFold
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from numpy.random import seed
import random
import time
import os
import math
from matplotlib import cm
import matplotlib 
import matplotlib.patches as patches

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

random.seed(1)
seed(2)
tf.random.set_seed(3)

def reset_seeds():
    np.random.seed(1)
    random.seed(2)
    tf.random.set_seed(3)

VERBOSE = 0

SPARSE_RESULT = []
SPARSE_RESULT_OPTIMIZED = []

SPARSE_COST = []
SPARSE_COST_OPTIMIZED = []

### Hyperparameter space definition

In [71]:
hyperparameterspace = {
    'x0': ["interval", -5, 10],
    'x1': ["interval", -5, 10],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste



### Experiment parameters

In [72]:


BUDGETS = [1 + i * 52 for i in range(19)]

### Optimization

In [73]:

################## MODEL AND FUNCTION DEFINITION ####################

def evaluate_model(epochs, batch_size):
    #return epochs * batch_size
    ################### ROSENBROCK ###############
    sum = 0
    sum += (1-epochs)**2 + 100 * (batch_size - epochs**2) ** 2
    return sum

    # ################# RASTRIGIN #################
    sum = 0
    sum += epochs ** 2 - 10 * math.cos(2 * math.pi * epochs)
    sum += batch_size ** 2 - 10 * math.cos(2 * math.pi * batch_size)
    
    return len(hyperparameterspace) * 10 + sum

def function(coordinates):
    return evaluate_model(coordinates[0], coordinates[1])


BUDGET = 400
INIT_POINTS = 5

print("INITIAL POINTS:")

reset_seeds()

optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, INIT_POINTS, 0, 0.5, init_points=INIT_POINTS, alternative=2)
points = optimizer.fit()

smallest_value = points[0].get_value()
index = 0
for j in range(len(points)):
    print("Point:", j)
    print(points[j].get_coordinates())
    if points[j].get_value() < smallest_value:
        smallest_value = points[j].get_value()
        index = j

print("Optimum:")
print(points[index].get_coordinates())
print(points[index].get_value())

x_values = []
y_values = []
z_values = [1, 1, 1, 1, 1]

for i in range(len(points)):
    x_values.append(points[i].get_coordinates()[0])
    y_values.append(points[i].get_coordinates()[1])


fig = plt.figure()
ax = plt.axes()
surface = plt.scatter(x_values, y_values, c="red")
plt.scatter([4.089162485176937], [4.1020260046125685], c="blue", marker="x")
plt.plot([-5, 10], [-0.3779531361628372, -0.3779531361628372], color='blue')
plt.plot([-5, 10], [6.039549836027849, 6.039549836027849], color='blue')
plt.plot([-4.15172948409787, -4.15172948409787], [-5, 10], color='blue')
plt.plot([5.045956021603313, 5.045956021603313], [-5, 10], color='blue')


rect = patches.Rectangle((-4.15172948409787, -0.3779531361628372), 5.045956021603313+4.15172948409787, 6.039549836027849+0.3779531361628372, linewidth=1, edgecolor='blue', facecolor='blue', alpha=0.16)

# Add the patch to the Axes
ax.add_patch(rect)

ax.set_xlim([-5, 10])
ax.set_ylim([-5, 10])



plt.gca().set_aspect("equal")

plt.savefig("./Alternative_1.pgf",bbox_inches='tight' )

plt.show()


INITIAL POINTS:
Point: 0
[4.089162485176937, 4.1020260046125685]
Point: 1
[-4.15172948409787, -3.7269200726161755]
Point: 2
[5.045956021603313, -0.3779531361628372]
Point: 3
[7.5324831719417435, 6.039549836027849]
Point: 4
[9.34051407833874, 9.21741230589024]
Optimum:
[4.089162485176937, 4.1020260046125685]
15934.023920832717


/tmp/ipykernel_25638/2350682644.py:77: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
